In [42]:
import requests
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta


In [84]:
API_KEYS = {
    "alpha_vantage": "1WD02A0ZXUENB863",
    "fred": "aadcef49fb39a772a54e92142ac7f12c",
    "api_ninjas": "ezhu6DmffTA5ORjPxHLMlQ==B8XXAjW8KBUDzijV",
    "open_exchange_rates": "f00b5c7db237401eb9964d25fc5206f0"
}


In [44]:
def fetch_data(api_url, csv_file, columns, json_key=None):
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            if json_key:  
                data = data[json_key]
            df = pd.DataFrame(data)
            return df[columns]
        else:
            print(f"⚠️ API failed ({response.status_code}), loading from {csv_file}")
    except Exception as e:
        print(f"⚠️ API error: {e}, loading from {csv_file}")

    return pd.read_csv(csv_file, parse_dates=["Date"])


In [47]:
api_url = f"https://openexchangerates.org/api/latest.json?app_id={API_KEYS['open_exchange_rates']}"
response = requests.get(api_url)
exchnage_rate = response.json()
usd_inr = data["rates"]["INR"]
print("✅ USD/INR Exchange Rate:", usd_inr)


✅ USD/INR Exchange Rate: 85.817413


In [54]:
# nifty_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NSE:NIFTY50&apikey={API_KEYS['alpha_vantage']}"
# sp500_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=SPY&apikey={API_KEYS['alpha_vantage']}"

# response = requests.get(nifty_url)
# data = response.json()
# latest_date = list(data["Time Series (Daily)"].keys())[0]
# nifty_value = float(data["Time Series (Daily)"][latest_date]["4. close"])
# print("✅ NIFTY:", nifty_value)


In [53]:
def get_nifty50_price():
    """Fetch the latest NIFTY 50 closing price from NSE India."""
    try:
        nse_url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
        headers = {
            "User-Agent": "Mozilla/5.0",
            "Accept-Language": "en",
            "Accept-Encoding": "gzip, deflate, br"
        }
        response = requests.get(nse_url, headers=headers)
        data = response.json()

        for item in data["data"]:
            if item["symbol"] == "NIFTY 50":
                return item["lastPrice"]

    except Exception as e:
        print(f"⚠️ Error fetching NIFTY 50 data: {e}")

    return None

# Test the function
nifty_price = get_nifty50_price()
print("✅ Latest NIFTY 50 Price:", nifty_price)


✅ Latest NIFTY 50 Price: 23592.9


In [56]:
def get_sp500_price():
    """Fetch the latest S&P 500 closing price from Alpha Vantage."""
    alpha_vantage_api_key = "1WD02A0ZXUENB863"
    sp500_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=SPY&apikey={alpha_vantage_api_key}"

    try:
        response = requests.get(sp500_url)
        data = response.json()

        latest_date = list(data["Time Series (Daily)"].keys())[0]  # Get most recent date
        return float(data["Time Series (Daily)"][latest_date]["4. close"])

    except Exception as e:
        print(f"⚠️ Error fetching S&P 500 data: {e}")
        return None

# Test the function
sp500_price = get_sp500_price()
print("✅ Latest S&P 500 Price:", sp500_price)


✅ Latest S&P 500 Price: 568.59


In [58]:
usa_interest_url = f"https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key={API_KEYS['fred']}&file_type=json"
india_interest_url = f"https://api.stlouisfed.org/fred/series/observations?series_id=INTDSRINM193N&api_key={API_KEYS['fred']}&file_type=json"

response = requests.get(usa_interest_url)
data = response.json()
usa_interest_rate = float(data["observations"][-1]["value"])
print("✅ Interest Rate USA:", usa_interest_rate)

response = requests.get(india_interest_url)
data = response.json()
india_interest_rate = float(data["observations"][-1]["value"])
print("✅ Interest Rate India:", india_interest_rate)

✅ Interest Rate USA: 4.33
✅ Interest Rate USA: 5.15


In [64]:
import requests

# ✅ Dictionary to Store API Keys
API_KEYS = {
    "fred": "aadcef49fb39a772a54e92142ac7f12c"
}

# ✅ API URLs for GDP Growth Rates
gdp_usa_url = f"https://api.stlouisfed.org/fred/series/observations?series_id=A191RL1Q225SBEA&api_key={API_KEYS['fred']}&file_type=json"
gdp_india_url = "http://api.worldbank.org/v2/country/IN/indicator/NY.GDP.MKTP.KD.ZG?format=json"

# 📌 Fetch India GDP Growth Rate
response = requests.get(gdp_india_url)
data = response.json()

# ✅ Extract GDP Growth Rate for India
if len(data) > 1 and "value" in data[1][-1]:
    gdp_india = float(data[1][-1]["value"])
    print("✅ GDP Rate India:", gdp_india)
else:
    print("⚠️ No data found for India GDP.")

# 📌 Fetch USA GDP Growth Rate
response = requests.get(gdp_usa_url)
data = response.json()

# ✅ Extract GDP Growth Rate for USA
if "observations" in data and len(data["observations"]) > 0:
    gdp_usa = float(data["observations"][-1]["value"])
    print("✅ GDP Rate USA:", gdp_usa)
else:
    print("⚠️ No data found for USA GDP.")


✅ GDP Rate India: 1.18533625466961
✅ GDP Rate USA: 2.3


In [76]:
# ✅ API URLs for Inflation Rates
inflation_usa_url = f"https://api.stlouisfed.org/fred/series/observations?series_id=FPCPITOTLZGUSA&api_key={API_KEYS['fred']}&file_type=json"
inflation_india_url = "http://api.worldbank.org/v2/country/IN/indicator/FP.CPI.TOTL.ZG?format=json"

# ✅ Fetch USA Inflation Rate (FRED)
response = requests.get(inflation_usa_url)
data = response.json()

if "observations" in data and len(data["observations"]) > 0:
    inflation_usa = float(data["observations"][-1]["value"])
    print("✅ Inflation Rate USA:", inflation_usa)
else:
    print("⚠️ No data found for USA Inflation Rate.")

# ✅ Fetch India Inflation Rate (World Bank)
response = requests.get(inflation_india_url)
data = response.json()

if len(data) > 1 and "value" in data[1][-1]:
    inflation_india = float(data[1][-1]["value"])
    print("✅ Inflation Rate India:", inflation_india)
else:
    print("⚠️ No data found for India Inflation Rate.")


✅ Inflation Rate USA: 4.11633838374488
✅ Inflation Rate India: 28.598734077509


In [78]:
dollar_index_url = f"https://api.stlouisfed.org/fred/series/observations?series_id=DTWEXM&api_key={API_KEYS['fred']}&file_type=json"

response = requests.get(dollar_index_url)
data = response.json()
dollar_index = float(data["observations"][-1]["value"])
print("✅ Dollar Index:", dollar_index)


✅ Dollar Index: 90.8221


In [85]:
unemployment_usa_url = f"https://api.stlouisfed.org/fred/series/observations?series_id=UNRATE&api_key={API_KEYS['fred']}&file_type=json"
unemployment_india_url = "http://api.worldbank.org/v2/country/IN/indicator/SL.UEM.TOTL.ZS?format=json"

# ✅ Fetch USA Unemployment Rate from FRED
response = requests.get(unemployment_usa_url)
data = response.json()

if "observations" in data and len(data["observations"]) > 0:
    unemployment_usa = float(data["observations"][-1]["value"])
    print("✅ Unemployment Rate USA:", unemployment_usa)
else:
    print("⚠️ No data found for USA Unemployment Rate.")

# ✅ Fetch India Unemployment Rate from World Bank
response = requests.get(unemployment_india_url)
data = response.json()

# Ensure the response contains valid data
if len(data) > 1 and isinstance(data[1], list) and len(data[1]) > 0:
    for entry in reversed(data[1]):  # Check latest available data
        if entry.get("value") is not None:
            unemployment_india = float(entry["value"])
            print("✅ Unemployment Rate India:", unemployment_india)
            break
    else:
        print("⚠️ No valid data found for India Unemployment Rate.")
else:
    print("⚠️ No data found for India Unemployment Rate.")


✅ Unemployment Rate USA: 4.1
✅ Unemployment Rate India: 7.722


In [87]:
import pandas as pd

# Load dataset
df = pd.read_csv("cleaned_dataset.csv", parse_dates=["Date"])
df.sort_values(by="Date", inplace=True)

# Lag Features
df["Exchange_Rate_Lag_1"] = df["Exchange Rate (USD/INR)"].shift(1)
df["Exchange_Rate_Lag_3"] = df["Exchange Rate (USD/INR)"].shift(3)
df["NIFTY_Lag_1"] = df["NIFTY"].shift(1)
df["SP500_Lag_1"] = df["S&P500"].shift(1)

# Moving Average
df["Exchange_Rate_MA_3"] = df["Exchange Rate (USD/INR)"].rolling(window=3).mean()

# Volatility (Standard Deviation over 7 days)
df["Exchange_Rate_Volatility_7"] = df["Exchange Rate (USD/INR)"].rolling(window=7).std()

# Percentage Change
df["Exchange_Rate_Percent_Change"] = df["Exchange Rate (USD/INR)"].pct_change() * 100
df["NIFTY_Percent_Change"] = df["NIFTY"].pct_change() * 100
df["SP500_Percent_Change"] = df["S&P500"].pct_change() * 100
df["GDP_India_Change"] = df["GDP Rate India"].pct_change() * 100
df["GDP_USA_Change"] = df["GDP Rate USA"].pct_change() * 100
df["Dollar_Index_Percent_Change"] = df["Dollar_Index"].pct_change() * 100

# Handling missing values after feature engineering
df.dropna(inplace=True)

# Save processed dataset
df.to_csv("processed_data.csv", index=False)

print("✅ Feature engineering completed and saved to 'processed_data.csv'")

✅ Feature engineering completed and saved to 'processed_data.csv'


In [97]:
import pandas as pd
import numpy as np
import requests

# Load the existing cleaned dataset
file_path = "cleaned_dataset.csv"
df = pd.read_csv(file_path, parse_dates=["Date"], dayfirst=True)

# Fetch the latest exchange rate data from the API
latest_exchange_rate = usd_inr

# Calculate exchange rate-based features using historical data
exchange_rate_series = df["Exchange Rate (USD/INR)"].copy()
exchange_rate_series = pd.concat([exchange_rate_series, pd.Series([latest_exchange_rate])], ignore_index=True)

df["Exchange_Rate_Lag_1"] = exchange_rate_series.shift(1)
df["Exchange_Rate_Lag_3"] = exchange_rate_series.shift(3)
df["Exchange_Rate_MA_3"] = exchange_rate_series.rolling(window=3).mean()
df["Exchange_Rate_Volatility_7"] = exchange_rate_series.pct_change().rolling(window=7).std()
df["Exchange_Rate_Percent_Change"] = exchange_rate_series.pct_change()

# Fetch other economic indicators from API (Replace these with actual API responses)
nifty = nifty_price  # Latest NIFTY Index value
sp500 = sp500_price   # Latest S&P 500 Index value
interest_rate_usa = usa_interest_rate
interest_rate_india = india_interest_rate
inflation_rate_india = inflation_india
inflation_rate_usa = inflation_usa
gdp_rate_india = gdp_india
gdp_rate_usa = gdp_usa
gdp_india_change = gdp_rate_india - df["GDP Rate India"].iloc[-1]
gdp_usa_change = gdp_rate_usa - df["GDP Rate USA"].iloc[-1]
exports = df["Exports"].iloc[-1]
imports = df["GDP Rate India"].iloc[-1]
unemployment_rate_india = unemployment_india
us_unemployment_rate = unemployment_usa
dollar_index = dollar_index
dollar_index_percent_change = (dollar_index - df["Dollar_Index"].iloc[-1]) / df["Dollar_Index"].iloc[-1]

# Create a new row with the latest data
new_row = pd.DataFrame({
    "Date": [pd.Timestamp.today().normalize()],
    "Exchange Rate (USD/INR)": [latest_exchange_rate],
    "Exchange_Rate_Lag_1": [df["Exchange_Rate_Lag_1"].iloc[-1]],
    "Exchange_Rate_Lag_3": [df["Exchange_Rate_Lag_3"].iloc[-1]],
    "Exchange_Rate_MA_3": [df["Exchange_Rate_MA_3"].iloc[-1]],
    "Exchange_Rate_Volatility_7": [df["Exchange_Rate_Volatility_7"].iloc[-1]],
    "Exchange_Rate_Percent_Change": [df["Exchange_Rate_Percent_Change"].iloc[-1]],
    "NIFTY": [nifty],
    "S&P500": [sp500],
    "NIFTY_Lag_1": [df["NIFTY"].iloc[-1]],
    "SP500_Lag_1": [df["S&P500"].iloc[-1]],
    "NIFTY_Percent_Change": [(nifty - df["NIFTY"].iloc[-1]) / df["NIFTY"].iloc[-1]],
    "SP500_Percent_Change": [(sp500 - df["S&P500"].iloc[-1]) / df["S&P500"].iloc[-1]],
    "Interest Rate USA": [interest_rate_usa],
    "Interest Rate India": [interest_rate_india],
    "Inflation_Rate_India": [inflation_rate_india],
    "Inflation_Rate_US": [inflation_rate_usa],
    "GDP Rate India": [gdp_rate_india],
    "GDP Rate USA": [gdp_rate_usa],
    "GDP_India_Change": [gdp_india_change],
    "GDP_USA_Change": [gdp_usa_change],
    "Exports": [exports],
    "Imports": [imports],
    "Unemployment_Rate_India": [unemployment_rate_india],
    "US_Unemployment_Rate": [us_unemployment_rate],
    "Dollar_Index": [dollar_index],
    "Dollar_Index_Percent_Change": [dollar_index_percent_change]
})

# Check if the latest date already exists
# print(new_row["Date"].iloc[0])
if latest_date != new_row["Date"].iloc[0]:
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv("processed_data.csv", index=False)
    print("✅ New data appended and saved as processed_data.csv")
else:
    print("⚠️ Latest date already exists. No new data added.")


✅ New data appended and saved as processed_data.csv


In [127]:
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

scaler_X = joblib.load('standard_X.pkl') 
scaler_y = joblib.load('standard_y.pkl')  
minmax_X = joblib.load('minmax_X.pkl')
minmax_y = joblib.load('minmax_y.pkl')

In [128]:
if 'Date' in new_row.columns:
    new_row = new_row.drop(columns=['Date'])

In [129]:
X_new = new_row.drop(columns=["Exchange Rate (USD/INR)"])
y_new = latest_exchange_rate

In [131]:
X_new.columns

Index(['Exchange_Rate_Lag_1', 'Exchange_Rate_Lag_3', 'Exchange_Rate_MA_3',
       'Exchange_Rate_Volatility_7', 'Exchange_Rate_Percent_Change', 'NIFTY',
       'S&P500', 'NIFTY_Lag_1', 'SP500_Lag_1', 'NIFTY_Percent_Change',
       'SP500_Percent_Change', 'Interest Rate USA', 'Interest Rate India',
       'Inflation_Rate_India', 'Inflation_Rate_US', 'GDP Rate India',
       'GDP Rate USA', 'GDP_India_Change', 'GDP_USA_Change', 'Exports',
       'Imports', 'Unemployment_Rate_India', 'US_Unemployment_Rate',
       'Dollar_Index', 'Dollar_Index_Percent_Change'],
      dtype='object')

In [141]:
ridge_model = joblib.load('best_ridge_model.pkl') 
X_new_scaled = scaler_X.transform(X_new)

y_pred_new = ridge_model.predict(X_new_scaled)
y_pred_new = y_pred_new.reshape(-1, 1)

y_pred_new_original = scaler_y.inverse_transform(y_pred_new)[0][0]

print("Regression: ")
print("Original: ", y_new)
print("Predictions: ", y_pred_new_original)

Regression: 
Original:  85.817413
Predictions:  755.9688567666287


In [144]:
xgboost_model = joblib.load('best_xgb_model.pkl')

y_pred_new = xgboost_model.predict(X_new)[0]
# y_pred_new = y_pred_new.reshape(-1, 1)

# y_pred_new_original = scaler_y.inverse_transform(y_pred_new)[0][0]

print("XG Boost: ")
print("Original: ", y_new)
print("Predictions: ", y_pred_new)

XG Boost: 
Original:  85.817413
Predictions:  82.76837


In [152]:
from keras.models import load_model
lstm_model = load_model('lstm_exchange_rate_model.h5')

In [153]:
X_new_scaled = X_new_scaled.reshape((X_new_scaled.shape[0], X_new_scaled.shape[1], 1))

# Make predictions using the LSTM model
y_pred_new_lstm = lstm_model.predict(X_new_scaled)

# Reshape predictions (if needed) to match the expected output format
y_pred_new_lstm = y_pred_new_lstm.reshape(-1, 1)

# Inverse transform predictions to get back to the original scale
y_pred_new_lstm_original = minmax_y.inverse_transform(y_pred_new_lstm)[0][0]

# Print results
print("LSTM Predictions:")
print("Original:", y_new)
print("Predictions:", y_pred_new_lstm_original)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step
LSTM Predictions:
Original: 85.817413
Predictions: 60.556797
